## Zillow Time Series Data

## Import libraries

In [52]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose

## Data Collection

In [53]:
zillow_df = pd.read_csv('../data/raw/zillow_data.csv')

In [54]:
zillow_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [55]:
len(zillow_df['RegionID'].unique())==len(zillow_df)

True

In [56]:
len(zillow_df['City'].unique())

7554

In [57]:
chicago_df = zillow_df[zillow_df['City']=='Chicago']

In [58]:
zillow_dates = chicago_df[chicago_df.columns[7:]]
zillow_dates.head()

,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,344400.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
3,498100.0,500900.0,503100.0,504600.0,505500.0,505700.0,505300.0,504200.0,503600.0,503400.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
7,216500.0,216700.0,216900.0,217000.0,217100.0,217200.0,217500.0,217900.0,218600.0,219700.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
16,122700.0,122800.0,122800.0,122700.0,122400.0,122000.0,121500.0,120900.0,120500.0,120300.0,...,459600,461400,464300,466500,467900,470600,474500,475100,472600,470200
19,142600.0,143100.0,143400.0,143300.0,142900.0,142200.0,141300.0,140100.0,139000.0,138000.0,...,462700,461200,459900,459200,458700,457900,457400,459000,462500,464300


In [61]:
zillow_T = zillow_dates.T
zillow_T.reset_index(inplace=True)

In [62]:
chicago_index = zillow_df.index[zillow_df['City']=='Chicago']
chicago_index

Int64Index([   0,    3,    7,   16,   19,   40,   48,   50,   74,   79,  105,
             150,  157,  215,  231,  284,  318,  324,  332,  347,  497,  638,
             658,  739,  792, 1012, 1360, 1454, 1496, 1727, 1990, 2360, 2420,
            2585, 2830, 3149, 3156, 3960, 4040, 4362, 7863],
           dtype='int64')

In [63]:
chicago_index = chicago_index.astype(str)

In [74]:
zillow_T.rename(columns={'index':'date'}, inplace=True)
zillow_T.head()

,date,0,3,7,16,19,40,48,50,74,...,2360,2420,2585,2830,3149,3156,3960,4040,4362,7863
0,1996-04-01,334200.0,498100.0,216500.0,122700.0,142600.0,297900.0,93400.0,148900.0,76400.0,...,100900.0,215600.0,165400.0,103700.0,180900.0,166300.0,125700.0,197300.0,224100.0,77900.0
1,1996-05-01,335400.0,500900.0,216700.0,122800.0,143100.0,300400.0,94100.0,149300.0,77000.0,...,101400.0,215100.0,165300.0,104400.0,182200.0,166900.0,126300.0,198900.0,223300.0,78200.0
2,1996-06-01,336500.0,503100.0,216900.0,122800.0,143400.0,302600.0,94600.0,149600.0,77400.0,...,101900.0,214500.0,165000.0,104700.0,183100.0,167200.0,126600.0,200100.0,222400.0,78300.0
3,1996-07-01,337600.0,504600.0,217000.0,122700.0,143300.0,304700.0,94800.0,149700.0,77700.0,...,102300.0,213900.0,164600.0,104700.0,183500.0,167100.0,126500.0,200800.0,221500.0,78500.0
4,1996-08-01,338500.0,505500.0,217100.0,122400.0,142900.0,306600.0,94700.0,149700.0,77700.0,...,102700.0,213200.0,164100.0,104400.0,183400.0,166900.0,126100.0,201000.0,220500.0,78500.0


In [73]:
zillow_T['date'] = pd.to_datetime(zillow_T['date'], unit='s')
# zillow_T.info()

In [78]:
chicago_zips = zillow_df.RegionName.iloc[chicago_index.astype(int)]
chicago_zips.head()

0     60657
3     60614
7     60640
16    60647
19    60618
Name: RegionName, dtype: int64

In [85]:
zillow_T.columns = ['date']+list(chicago_zips)
zillow_chicago = zillow_T
zillow_chicago.head()

,date,60657,60614,60640,60647,60618,60613,60629,60625,60619,...,60612,60607,60659,60652,60631,60656,60655,60646,60642,60633
0,1996-04-01,334200.0,498100.0,216500.0,122700.0,142600.0,297900.0,93400.0,148900.0,76400.0,...,100900.0,215600.0,165400.0,103700.0,180900.0,166300.0,125700.0,197300.0,224100.0,77900.0
1,1996-05-01,335400.0,500900.0,216700.0,122800.0,143100.0,300400.0,94100.0,149300.0,77000.0,...,101400.0,215100.0,165300.0,104400.0,182200.0,166900.0,126300.0,198900.0,223300.0,78200.0
2,1996-06-01,336500.0,503100.0,216900.0,122800.0,143400.0,302600.0,94600.0,149600.0,77400.0,...,101900.0,214500.0,165000.0,104700.0,183100.0,167200.0,126600.0,200100.0,222400.0,78300.0
3,1996-07-01,337600.0,504600.0,217000.0,122700.0,143300.0,304700.0,94800.0,149700.0,77700.0,...,102300.0,213900.0,164600.0,104700.0,183500.0,167100.0,126500.0,200800.0,221500.0,78500.0
4,1996-08-01,338500.0,505500.0,217100.0,122400.0,142900.0,306600.0,94700.0,149700.0,77700.0,...,102700.0,213200.0,164100.0,104400.0,183400.0,166900.0,126100.0,201000.0,220500.0,78500.0


In [86]:
zillow_chicago.to_csv('../data/processed/zillow_chicago_df.csv')